# Practical 4 - Seismic attribute analysis

This notebook aims to build on your investigation of 3D seismic volumes from practical 3 by providing methods which we can use to enhance the identification of certain geological/environmental features. This is achieved through seismic attributes, attributes use some mathematical calculation of the data to enhance features, very similar to image processing techniques. 

**Learning Objectives**:

1)	Understand that modern seismic data analysis involves calculating seismic attributes to reveal features more clearly
2)	Understand that new attributes can be designed or multiple attributes combined to improve imaging
3)	Give you an appreciation that modern seismic data analysis will require the development of more sophisticated data analysis techniques


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import segyio


In [3]:
import itertools
import scipy.ndimage

In [4]:
import pandas as pd

In [ ]:


from mayavi import mlab

### 4.1 Explore a new 3D seismic data volume

We are going to start this exercise by loading a new 3D seismic data volume- Thebe_rift_faults. As we discussed in  lecture 6 this dataset comes from offshore NW Australia and shows some great examples of faults. This seismic datsets has time as the z-axis (ie it has not been depth converted yet).

In [ ]:
base_segy = 'data/Thebe_rift_faults'
f = segyio.open(base_segy, ignore_geometry= True) #opens files like python default open call , assumes a well-structured 3D volume, 

In [ ]:
print (segyio.tools.wrap(f.text[0])) #show text header to understand file better

In [ ]:
data = segyio.tools.cube(base_segy) #extremely easy for well structured data

In [ ]:
def explore3d(data_cube):
    """
    visualise data volume
    """
    source = mlab.pipeline.scalar_field(data_cube)
    source.spacing = [1, 1, -1]
    vm = np.percentile(data_cube, 95) #may need to play a little with the 95

    nx, ny, nz = data_cube.shape
    mlab.pipeline.image_plane_widget(source, plane_orientation='x_axes', 
                                     slice_index=nx//2, colormap='coolwarm', vmin=-vm, vmax=vm)
    mlab.pipeline.image_plane_widget(source, plane_orientation='y_axes', 
                                     slice_index=ny//2, colormap='coolwarm', vmin=-vm, vmax=vm)
    mlab.pipeline.image_plane_widget(source, plane_orientation='z_axes', 
                                     slice_index=nz//2, colormap='coolwarm', vmin=-vm, vmax=vm)
    mlab.show()

In [ ]:
explore3d(data)

**Ex 1: Spend some time looking through this data volume like you did in Exercise 3. Do you identify any interesting features that you recognise from the lecture 5 notes? If so, show some examples below:**

In the next part of the exercise you are going to use seismic attributes to try and highlight these geolgical features even further...

## 4.1.1. Attribute 1 - RMS amplitude

Amplitude attributes can be extracted along mapped horizons or from time/depth windows. RMS amplitude is just one example of an amplitude attribute. In this exercise we are going to calculate RMS between a time window. This involves squaring the amplitudes within the window and summing them, then square rooting. 

https://wiki.seg.org/wiki/RMS_amplitude

In [ ]:
RMS = np.zeros(data.shape)

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        trace = data[i,j,:]
        
        # Get RMS values
        trace_sq = np.square(trace) # square 
        kernel = np.ones(11)/11 # range over which we take the RMS value
        
        # complete RMS calc
        RMS_amp = np.sqrt(np.convolve(trace_sq, kernel, 'same'))
        
        #store results
        RMS[i,j,:] = RMS_amp

In [ ]:
explore3d(RMS)

## 4.1.2. Attribute 2 - 1st vertical derivative in time

Any calculation you do with seismic data is a seismic attribute! New seismic attributes are invented all the time. You could invent your own! Some are really useful, and some less so... In this part of the exercise we will calcualte an attribute which is often found in seismic interpretation software- the 1st vertical derivative. This will highlight vertical gradients. Do you find this attribute highlights features any differently?

https://primer-computational-mathematics.github.io/book/d_geosciences/remote_sensing/Image_Filtering.html

In [ ]:
diff1 = np.zeros(data.shape)
dt = (f.samples[1]-f.samples[0])

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        trace = data[i,j,:]

        kernel = np.array([-1,1])/dt  #forward difference https://en.wikipedia.org/wiki/Finite_difference
        
        # complete RMS calc
        diff = np.convolve(trace, kernel, 'same')
        
        #store results
        diff1[i,j,:] = diff

In [ ]:
explore3d(diff1)

Please note the effect that our vm parameter has here when optimising our dynamic range! If you use the same dynamic range for both the effects become a lot clearer!

In [ ]:
fig = plt.figure(figsize=(20,15))
ax1 = plt.subplot(121)
vm = np.percentile(data, 95)
ax1.imshow(data[0,:,:].T, cmap='coolwarm', aspect='auto', vmin=-vm, vmax=vm)
ax2 = plt.subplot(122)
vm = np.percentile(diff1, 95)
ax2.imshow(diff1[0,:,:].T, cmap='coolwarm', aspect='auto', vmin=-vm, vmax=vm)
plt.show()

## 4.1.3.  Attribute 3 - 2nd vertical derivative in time

...again, we can do any calculation we like with the data to design a new attribute. Do you find this one very helpful? Do any features become sharper/clearer?

In [ ]:
diff2 = np.zeros(data.shape)
dt = (f.samples[1]-f.samples[0])

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        trace = data[i,j,:]

        kernel = np.array([1,-2,1])/ (dt**2)  #second order central https://en.wikipedia.org/wiki/Finite_difference
        
        # complete RMS calc
        diff = np.convolve(trace, kernel, 'same')
        
        #store results
        diff2[i][j][:] = diff

In [ ]:
explore3d(diff2)

In [ ]:
fig = plt.figure(figsize=(20,15))
ax1 = plt.subplot(121)
vm = np.percentile(data, 95)
ax1.imshow(data[0,:,:].T, cmap='coolwarm', aspect='auto', vmin=-vm, vmax=vm)
ax2 = plt.subplot(122)
vm = np.percentile(diff2, 95)
ax2.imshow(diff2[0,:,:].T, cmap='coolwarm', aspect='auto', vmin=-vm, vmax=vm)
plt.show()

## 4.1.4.  Atrribute 4- Coherency

As we discussed in lecture 6, discontinuity attributes are really important, and the algorithms to compute them are often trade-secrets in seismic interpretation software. In this part of the exercise we are using some code from a Github tutorial which shows a number of ways to calculate discontinuity attributes- https://github.com/seg/tutorials-2015/blob/master/1512_Semblance_coherence_and_discontinuity/writeup.md

In this exercise we are going to use the "marfurt semblance". Semblance (also known as coherence) is a measure of how similar seismic traces are to each other. This code is slow... so we are only going to calculate coherence for a few time slices. 


In [ ]:
def moving_window(data, window, func):
    # `generic_filter` will give the function 1D input. We'll reshape it for convinence
    wrapped = lambda region: func(region.reshape(window))
    
    # Instead of an explicit for loop, we'll use a scipy function to do the same thing
    # The boundaries will be handled by "reflecting" the data, by default
    return scipy.ndimage.generic_filter(data, wrapped, window)

def marfurt_semblance(region):
    # We'll need an ntraces by nsamples array
    # This stacks all traces within the x-y "footprint" into one
    # two-dimensional array.
    region = region.reshape(-1, region.shape[-1])
    ntraces, nsamples = region.shape

    square_of_sums = np.sum(region, axis=0)**2
    sum_of_squares = np.sum(region**2, axis=0)
    sembl = square_of_sums.sum() / sum_of_squares.sum()
    return sembl / ntraces

Due to the processing cost of coherency we will only be running this on 15 time-slices. You can run this for more layers if you like, but it may take a while. An applied eaxmple is shown below.

In [ ]:
# due to processing cost we only run this on 15 layers
marfurt = moving_window(data[:,:,100:115], (3, 3, 9), marfurt_semblance)

In [ ]:
fig = plt.figure(figsize=(15,8))
ax1 = plt.subplot(121)
vm = np.percentile(marfurt, 95)
plt.imshow(marfurt[:,:,4].T, cmap='gray', aspect='auto', vmin=-vm, vmax=vm)
plt.colorbar()
ax2 = plt.subplot(122)
vm = np.percentile(data, 95)
plt.imshow(data[:,:,104].T, cmap='gray', aspect='auto', vmin=-vm, vmax=vm)
plt.show()

**Ex 2: Does coherency help better highlight the features you observed in 4.1.1? If you were a seismic interpreter would you rather interpret these fetures in the amplitude time slices (ie time slices through the original seismic) or the coherence?**

**Ex 3 (optional): You could go a step further and take the derivative of the coherency volume above... remember, any calculation you do is a seismic attribute! You never know... you might design the next big seismic attribute!**

## 4.2. Attribute blending

It can be useful to overlay two or more attributes using transparency to highlight features even further- this is called multi-attribute analysis. Let's first visualise two different attributes side by side to understand visually what is ocurring with some of these attributes and which features they are enhancing

In [ ]:
def explore3d_2(data_cube1, data_cube2, cmap1 = 'gray', cmap2 = 'gray'):
    """
    code to visualise two data volumes together
    """
    # https://github.com/seg/tutorials-2015/blob/master/1512_Semblance_coherence_and_discontinuity/Discontinuity_tutorial.ipynb
    source1 = mlab.pipeline.scalar_field(data_cube1)
    source1.spacing = [1, 1, -1]
    
    source2 = mlab.pipeline.scalar_field(data_cube2)
    source2.spacing = [1, 1, -1]

    nx, ny, nz = data_cube1.shape
    
    vm = np.percentile(data_cube1, 95)
    mlab.pipeline.image_plane_widget(source1, plane_orientation='x_axes', 
                                     slice_index=nx//2, colormap= cmap1, vmin=-vm, vmax=vm, plane_opacity=0.1)
    mlab.pipeline.image_plane_widget(source1, plane_orientation='y_axes', 
                                     slice_index=ny//2, colormap= cmap1, vmin=-vm, vmax=vm, plane_opacity=0.1)
    mlab.pipeline.image_plane_widget(source1, plane_orientation='z_axes', 
                                     slice_index=nz//2, colormap= cmap1, vmin=-vm, vmax=vm, plane_opacity=0.1)
    
    vm = np.percentile(data_cube2, 95)
    mlab.pipeline.image_plane_widget(source2, plane_orientation='x_axes', 
                                     slice_index=nx//2, colormap= cmap2, vmin=-vm, vmax=10)
    mlab.pipeline.image_plane_widget(source2, plane_orientation='y_axes', 
                                     slice_index=ny//2, colormap= cmap2, vmin=-vm, vmax=vm)
    mlab.pipeline.image_plane_widget(source2, plane_orientation='z_axes', 
                                     slice_index=nz//2, colormap= cmap2, vmin=-vm, vmax=vm)
    
    mlab.show()

In [ ]:
explore3d_2(data, RMS)

An alternative way to use attributes together would be by simply plotting them over each other with the top layer with a higher transparency

In [ ]:
import matplotlib

In [ ]:
# lets create our own colorbar for clearer results
cmap = matplotlib.colors.ListedColormap(['red','white'])
bounds=[0,0.5,1]
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

In [ ]:
# plot two datasets together
vm = np.percentile(data, 95)
plt.imshow(data[:,:,104].T, cmap='gray', aspect='auto', vmin=-vm, vmax=vm)

vm = np.percentile(marfurt, 99)
plt.imshow(marfurt[:,:,4].T, cmap=cmap, aspect='auto', alpha = 0.5) # alpha controls transparency

plt.show()

Lastly, something commonly done in seismic interpretation is RGB blending. To achieve this every attribute is assigned either a Red, Blue, or Green colour and these layers and then stacked to produce a color image output that contains information of all 3 attributes. 

**EX 4: Repeat this exercise but with the TNW_small_2 datset we looked at in Exercise 3. Can you reveal any geological features better with attributes than you could with the original seismic volume in Exercise 3?**